# VishionDepthNet

Sample implementation of distance estimation in dashcam-like use cases.

## Environment setup

Install required libraries

In [ ]:
!pip install einops==0.8.1
!pip install matplotlib==3.10.0 matplotlib-inline==0.1.7 matplotlib-venn==1.1.2
!pip install numpy==2.0.2
!pip install opencv-contrib-python==4.11.0.86 opencv-python==4.11.0.86 opencv-python-headless==4.11.0.86
!pip install timm==1.0.15
!pip install ultralytics==8.3.102 ultralytics-thop==2.0.14

# Install PyTorch (compatible version for Colab, CPU/GPU auto)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

## Depth estimation

In this code block, we demonstrate how to utilize YOLO and MiDaS models for distance estimation. First, we define YOLO classes to identify (cars, bikes and pedestrians). Once the bounding boxes are detected, we apply MiDaS depth estimation map on the image that gives relative (!) distance distribution from the camera perspective. As one-shot distance estimation is generally a difficult task even for humans, we experimentally develop a formula to convert the depths to meters - feel free to play around with both the scaling factor and the formula itself.

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from ultralytics import YOLO
import matplotlib.pyplot as plt

# 1. Calibration parameters
SCALE_FACTOR = 600_000
CLASSES = [0, 1, 2]     # 0 = person, 1 = bike, 2 = car
FRAME_SIZE = (384, 640)

# 2. Detection and segmentation function
def detect_and_segment(image_input, conf_threshold=0.5):
    model = YOLO("yolov8s-seg.pt")  # Make sure this file is available
    results = model.predict(image_input, conf=conf_threshold, classes=CLASSES)

    detections = []
    for result in results:
        for i, box in enumerate(result.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            mask = result.masks[i].data[0].cpu().numpy()
            mask = cv2.resize(mask, (result.orig_shape[1], result.orig_shape[0]))

            detections.append({
                "bbox": (x1, y1, x2, y2),
                "mask": mask,
                "class": result.names[int(box.cls[0])],
                "confidence": box.conf[0].item()
            })

    return detections, results[0].orig_img

# 3. Depth model initialization
def setup_depth_estimator():
    model = torch.hub.load("intel-isl/MiDaS", "MiDaS_small", trust_repo=True)
    model.eval()

    transforms = Compose([
        Resize(FRAME_SIZE),
        ToTensor(),
        Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    return model, transforms

# 4. Depth estimation
def estimate_depth(image, model, transforms):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    input_tensor = transforms(img_pil).unsqueeze(0)

    with torch.no_grad():
        disparity = model(input_tensor)

    depth_map = disparity.squeeze().cpu().numpy()
    return depth_map

# 5. Distance calculation
def calculate_distances(detections, depth_map, original_image_shape):
    depth_map = cv2.resize(depth_map, (original_image_shape[1], original_image_shape[0]), interpolation=cv2.INTER_CUBIC)

    for det in detections:
        mask = det["mask"] > 0.5
        object_depths = depth_map[mask]

        if object_depths.size == 0:
            det["distance"] = None
            continue

        max_depth = np.max(object_depths)
        average_depth = np.mean(object_depths)
        det["distance"] = average_depth

        y, x = np.where((depth_map == max_depth) & mask)
        det["closest_point"] = (x[0], y[0]) if len(x) > 0 else None

    return detections

# 5.1 Convert depth map to meters
def convert_to_meters(depth_map):
    depth_map_meters = SCALE_FACTOR / (depth_map * depth_map)
    return depth_map_meters

# 6. Visualization
def visualize(image, detections, depth_map, save_path=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    display_image = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)

    for det in detections:
        if det["distance"] is None:
            continue

        mask = det["mask"] > 0.5
        overlay = display_image.copy()
        overlay[mask] = [0, 255, 255]
        cv2.addWeighted(overlay, 0.5, display_image, 0.5, 0, display_image)

        x1, y1, x2, y2 = det["bbox"]
        cv2.rectangle(display_image, (x1, y1), (x2, y2), (0,255,0), 2)
        print(f"{det['class']} {det['distance']:.1f}m")
        cv2.putText(display_image, f"{det['class']} {det['distance']:.1f}m", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        if det["closest_point"]:
            x, y = det["closest_point"]
            cv2.circle(display_image, (x, y), 8, (255,0,0), -1)

    if save_path:
        cv2.imwrite(save_path, cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR))
    else:
        ax1.imshow(display_image)
        ax1.set_title("Detections with masks and distances")
        ax2.imshow(depth_map, cmap="plasma")
        ax2.set_title("Depth map")
        plt.show()


## Sample usage of depth estimation

We extracted subsequent parts of code into functions so that the flow is clear. Below we present how to use it step by step.

In [ ]:
# Example usage

# Upload image if needed
#from google.colab import files
#uploaded = files.upload()

# Make sure you upload yolov8s-seg.pt model too
# Or you can download it directly inside Colab
# Example:
!wget https://github.com/ultralytics/assets/releases/download/v8.0.0/yolov8s-seg.pt

# 1. Detection + Segmentation
detections, image = detect_and_segment("testimage.jpg")
original_shape = image.shape[:2]

# 2. Depth model initialization
depth_model, depth_transforms = setup_depth_estimator()

# 3. Depth estimation
depth_map = estimate_depth(image, depth_model, depth_transforms)
depth_map_meters = convert_to_meters(depth_map)

# 4. Distance calculations
detections = calculate_distances(detections, depth_map_meters, original_shape)

# 5. Visualization
visualize(image, detections, depth_map)

## Car copilot

In the second part of the notebook we give a usage example to analyze frames of dashcam stream of the video in front of a moving car. Every frame is processed to detect pedestrians (example), find the closest one and estimate the velocity how quick the car approaches the pedestrian (based on FPS and distance difference between the two photos separated by number of frames, 30 in the code).

In [4]:
import os
import random
import cv2
import numpy as np

FPS = 30
FRAME_SIZE = (640, 448)

def road_to_stop(velocity, deceleration=9.8, tire_friction=0.7):
    effective_deceleration = deceleration * tire_friction
    return (velocity ** 2) / (2 * effective_deceleration)

def press_brake():
    print("🚨 Pressing the brake! 🚨")

def process_image(frame):
    frame = cv2.resize(frame, FRAME_SIZE)
    detections, image = detect_and_segment(frame)
    original_shape = image.shape[:2]

    depth_model, depth_transforms = setup_depth_estimator()

    depth_map = estimate_depth(image, depth_model, depth_transforms)
    depth_map_meters = convert_to_meters(depth_map)

    detections = calculate_distances(detections, depth_map_meters, original_shape)

    visualize(image, detections, depth_map_meters, save_path=None)  # Show image instead of saving

    min_distance = float('inf')

    for det in detections:
        if det["class"] == "person" and det["distance"] is not None:
            print(f"Detected {det['class']} at distance: {det['distance']:.2f} m")
            min_distance = min(det["distance"], min_distance)

    if min_distance == float('inf'):
        print("INFO: No person detected.")

    return min_distance

def car_copilot(dashcam_video_stream):
    context = 50
    distances = [0.0] * context
    current_index = 0
    frame_count = 0

    # Initialize once!
    depth_model, depth_transforms = setup_depth_estimator()

    cap = cv2.VideoCapture(dashcam_video_stream)
    if not cap.isOpened():
        raise ValueError(f"Unable to open video file: {dashcam_video_stream}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, FRAME_SIZE)

        # Important: Now pass the preloaded depth_model to avoid reloading every frame
        detections, image = detect_and_segment(frame)
        original_shape = image.shape[:2]
        depth_map = estimate_depth(image, depth_model, depth_transforms)
        depth_map_meters = convert_to_meters(depth_map)
        detections = calculate_distances(detections, depth_map_meters, original_shape)

        min_distance = float('inf')
        for det in detections:
            if det["class"] == "person" and det["distance"] is not None:
                min_distance = min(det["distance"], min_distance)

        if min_distance == float('inf'):
            current_distance = 9999  # No person detected, set far distance
        else:
            current_distance = min_distance

        if current_index < context:
            distances[current_index] = current_distance
        else:
            previous_distance = distances[current_index % context]
            delta_distance = current_distance - previous_distance
            velocity = delta_distance * FPS

            road_to_stop_distance = road_to_stop(velocity)
            print(f"Velocity: {velocity:.2f} m/s, Road distance to stop: {road_to_stop_distance:.2f} m")

            if road_to_stop_distance > current_distance:
                print("⚠️ Warning: Car is NOT in safe stopping distance!")
                press_brake()

        distances[current_index % context] = current_distance
        current_index += 1
        frame_count += 1

        print(f"Frame: {frame_count}, Distance to closest person: {current_distance:.2f} m")

    cap.release()
    return frame_count

## Sample usage

As before, you are free to play with the parameters or provide your own video. The usage of car copilot is easy from this point - just run the code block below.

In [ ]:
# Upload the dashcam video
#from google.colab import files
#uploaded = files.upload()

# Now, run the co-pilot!
car_copilot("testvideo.mp4")  # replace filename if needed


## Big thanks for the attention of yours, enjoy your time